# On copies and views in NumPy

For small data sets (< 1MB), modern computers are so amazingly fast that efficiency is hardly worth thinking about.
So, I'm assuming you are using bigger data sets that actually take some time to process, even on speedy machines.
For example, in the previous section, we allocated data arrays of 800MB and it took a couple of seconds to process it.
With that kind of data, speed starts to matter.

A big part of data processing speed is memory management.
Copying big arrays takes time, so the less of that we do, the faster our code runs.
The rules of when NumPy copies data are not trivial and it is worth your while to take a closer look at them.

## An example: matrix transpose

Transposing a matrix means that all rows become columns and all columns become rows. All off-diagonal values change places. Obviously, all this moving data around should take come time, right? Let's see how long NumPy's transpose function takes, by transposing a huge (10000 ✕ 10000) matrix. This time, let's exclude the time it takes to allocate this beast of a matrix and focus solely on the [np.transpose](https://docs.scipy.org/doc/numpy/reference/generated/numpy.transpose.html) function:

In [ ]:
import numpy as np
np.random.seed(0)
a = np.random.rand(10000, 10000)
print('Matrix `a` takes up %d MB' % (a.nbytes / 10**6))

In [ ]:
%%timeit -r 10 -n 1000
b = a.transpose()

<img src="images/magic.jpg" style="width: 200px; float: right; margin: 10px; margin-top: -10px"/>

On my laptop, it takes mere nanoseconds to transpose 800 MB of data.
NumPy somehow avoided copying the data.
The transpose operation is computationally so cheap that is also available through the `.T` property (it is conventially not done to expose methods that are potentially computationally expensive as properties).

What black magic is going on here?
To answer that, we must dive a little into the innards of NumPy arrays.

## The `ndarray` data structure